In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_california_housing
X = fetch_california_housing()
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(X.data, X.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [3]:


tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
norm_layer,
tf.keras.layers.Dense(100, activation="relu"),
tf.keras.layers.Dense(50, activation="relu"),
tf.keras.layers.Dense(50, activation="relu"),
tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=30,
validation_data=(X_valid, y_valid))
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)
model.evaluate(X_valid,y_valid)

Epoch 1/30
363/363 [==============================] - 1s 2ms/step - loss: 0.8103 - root_mean_squared_error: 0.9002 - val_loss: 0.3706 - val_root_mean_squared_error: 0.6088
Epoch 2/30
363/363 [==============================] - 0s 1ms/step - loss: 0.3698 - root_mean_squared_error: 0.6081 - val_loss: 3.0228 - val_root_mean_squared_error: 1.7386
Epoch 3/30
363/363 [==============================] - 0s 1ms/step - loss: 0.3733 - root_mean_squared_error: 0.6110 - val_loss: 0.3234 - val_root_mean_squared_error: 0.5687
Epoch 4/30
363/363 [==============================] - 0s 1ms/step - loss: 0.3391 - root_mean_squared_error: 0.5823 - val_loss: 0.7871 - val_root_mean_squared_error: 0.8872
Epoch 5/30
363/363 [==============================] - 0s 1ms/step - loss: 0.3261 - root_mean_squared_error: 0.5711 - val_loss: 1.3328 - val_root_mean_squared_error: 1.1545
Epoch 6/30
363/363 [==============================] - 0s 1ms/step - loss: 0.3242 - root_mean_squared_error: 0.5694 - val_loss: 0.5469 - val_

[0.44087955355644226, 0.6639876365661621]

In [6]:
import keras_tuner as kt
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
    sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
        model.add(tf.keras.layers.Dense(10, activation="softmax"))
        model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
        metrics=["accuracy"])
    return model